In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D
from tensorflow.keras.layers import Input, LeakyReLU, Softmax, Reshape, Flatten
from tensorflow.keras.layers import concatenate, maximum, Lambda, Layer
import pickle

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Additional layers not available in core Keras

In [2]:
class Bias(Layer):
    """
    Adds bias to a layer. This is used for untied biases convolution. 
    """
    def __init__(self, **kwargs):
        super(Bias, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.bias = self.add_weight(name='bias', 
                                      shape=input_shape[1:],
                                      initializer='uniform',
                                      trainable=True)
        super(Bias, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.add(x, self.bias)

    def compute_output_shape(self, input_shape):
        return input_shape

In [3]:
def feature_pool_max(input, pool_size=2, axis=1):
    """
    Based on lasagne implementation of FeaturePool
    """
    input_shape = input.shape.as_list()
    num_feature_maps = input_shape[axis]
    num_feature_maps_out = num_feature_maps // pool_size
    
    pool_shape = tf.TensorShape(
        (input_shape[1:axis] + [num_feature_maps_out, pool_size] + input_shape[axis+1:])
    )
    
    print(pool_shape)
    input_reshaped = Reshape(pool_shape)(input)
    # reduce along all axis but the target one
    reduction_axis = list(range(1, len(pool_shape)+1))
    reduction_axis.pop(axis-1)
    
    return tf.reduce_max(input_reshaped, axis=reduction_axis)

# create Layer for reshape with batchsize
# strong_reshape_func = lambda x: tf.reshape(x, (batch_size//2, concat.shape[1]*2))
# StrongReshape = Lambda(strong_reshape_func)

# Building the main model

In [4]:
def build_model(width=512, height=512, filename=None,
                n_classes=5, batch_size=64, p_conv=0.0):
    """
    
    """
    # Input shape (height, width, depth), different from original implimentation
    main_input = Input(shape=(height, width, 3),
                       batch_size=batch_size,
                      )
    
    # Note: for conv layers paper uses untie_biases=True
    # layer will have separate bias parameters for each position in each channel. 
    # As a result, the b attribute will be a 3D tensor.

    # TODO: set names of layers, check init
    # no need to init weights as they will be loaded from a file
    # Conv layers(filters, kernel_size)
    conv_main_1 = Conv2D(32, 7, strides=(2, 2), padding='same', 
                    use_bias=False,
                    activation= None,
                   )(main_input)
    conv_bias_1 = Bias()(conv_main_1)
    conv_activation_1 = LeakyReLU(alpha=0.5)(conv_bias_1)
    dropout_1 = Dropout(p_conv)(conv_activation_1)
    maxpool_1 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_1)
    # 3
    conv_main_2 = Conv2D(32, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation= None,
                        )(maxpool_1)
    conv_bias_2 = Bias()(conv_main_2)
    conv_activation_2 = LeakyReLU(alpha=0.5)(conv_bias_2)
    dropout_2 = Dropout(p_conv)(conv_activation_2)
    # 4
    conv_main_3 = Conv2D(32, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation= None,
                        )(dropout_2)
    conv_bias_3 = Bias()(conv_main_3)
    conv_activation_3 = LeakyReLU(alpha=0.5)(conv_bias_3)
    dropout_3 = Dropout(p_conv)(conv_activation_3)
    maxpool_3 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_3)
    # 6
    conv_main_4 = Conv2D(64, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_3)
    conv_bias_4 = Bias()(conv_main_4)
    conv_activation_4 = LeakyReLU(alpha=0.5)(conv_bias_4)
    dropout_4 = Dropout(p_conv)(conv_activation_4)
    # 7
    conv_main_5 = Conv2D(64, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_4)
    conv_bias_5 = Bias()(conv_main_5)
    conv_activation_5 = LeakyReLU(alpha=0.5)(conv_bias_5)
    dropout_5 = Dropout(p_conv)(conv_activation_5)
    maxpool_5 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_5)
    # 9 
    conv_main_6 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_5)
    conv_bias_6 = Bias()(conv_main_6)
    conv_activation_6 = LeakyReLU(alpha=0.5)(conv_bias_6)
    dropout_6 = Dropout(p_conv)(conv_activation_6)
    # 10    
    conv_main_7 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_6)
    conv_bias_7 = Bias()(conv_main_7)
    conv_activation_7 = LeakyReLU(alpha=0.5)(conv_bias_7)
    dropout_7 = Dropout(p_conv)(conv_activation_7)
    # 11
    conv_main_8 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_7)
    conv_bias_8 = Bias()(conv_main_8)
    conv_activation_8 = LeakyReLU(alpha=0.5)(conv_bias_8)
    dropout_8 = Dropout(p_conv)(conv_activation_8)
    # 12
    conv_main_9 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_8)
    conv_bias_9 = Bias()(conv_main_9)
    conv_activation_9 = LeakyReLU(alpha=0.5)(conv_bias_9)
    dropout_9 = Dropout(p_conv)(conv_activation_9)
    maxpool_9 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_9)
    # 14
    conv_main_10 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_9)
    conv_bias_10 = Bias()(conv_main_10)
    conv_activation_10 = LeakyReLU(alpha=0.5)(conv_bias_10)
    dropout_10 = Dropout(p_conv)(conv_activation_10)
    # 15
    conv_main_11 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_10)
    conv_bias_11 = Bias()(conv_main_11)
    conv_activation_11 = LeakyReLU(alpha=0.5)(conv_bias_11)
    dropout_11 = Dropout(p_conv)(conv_activation_11)
    # 16
    conv_main_12 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_11)
    conv_bias_12 = Bias()(conv_main_12)
    conv_activation_12 = LeakyReLU(alpha=0.5)(conv_bias_12)
    dropout_12 = Dropout(p_conv)(conv_activation_12)
    # 17
    conv_main_13 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_12)
    conv_bias_13 = Bias()(conv_main_13)
    conv_activation_13 = LeakyReLU(alpha=0.5)(conv_bias_13)
    dropout_13 = Dropout(p_conv)(conv_activation_13)
    maxpool_13 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_13)
    # 19, special dropout between phases with p=1/2 
    dropout_inter = Dropout(0.5)(maxpool_13)
    # 20 Dense phase 
    # Begins with Maxout layer, which is implemented here as Dense+custom feature_pool function
    maxout_1 = Dense(units=1024, 
                     activation=None,)(Flatten()(dropout_inter))
    # need to wrap operation in Lambda to count as a layer
    maxout_2 = Lambda(lambda x: feature_pool_max(x, pool_size=2, axis=1))(maxout_1)
    
    # 22 Concatenate with processed img, take both eyes into account
    img_dim_input = Input(shape=(2,), batch_size=batch_size, name="imgdim")
    concat = concatenate([maxout_2, img_dim_input], axis=1)
    
    # 24
    # flatten = Reshape((-1, concat.output_shape()[1] * 2))(concat)
    print(concat.shape)
    # use lambda for custom reshape that's capable of changing batch_size as well
    # expect order left-right
    # TODO: (-1, net['23'].output_shape[1] * 2)
    flatten = Lambda(lambda x: tf.reshape(x, (batch_size//2, concat.shape[1]*2)))(concat)
    dense_droupout_0 = Dropout(0.5)(flatten)
    # 26
    dense_1 = Dense(units=1024,
                    activation=None,
                   )(dense_droupout_0)
    dense_maxpool_1 = Lambda(lambda x: feature_pool_max(x, pool_size=2, axis=1))(dense_1)
    dense_dropout_1 = Dropout(0.5)(dense_maxpool_1)
    
    # 29
    dense_2 = Dense(units=n_classes*2,
                    activation=None,
                   )(dense_dropout_1)
    softmax_flatten = Lambda(lambda x: tf.reshape(x, (batch_size, n_classes)))(dense_2)
    softmax = keras.layers.Softmax()(softmax_flatten)
    
    model = Model(inputs=[main_input, img_dim_input], outputs=[softmax])
    
    return model

In [5]:
model = build_model()
model.summary()

(512, 2)
(64, 514)
(512, 2)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (64, 512, 512, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (64, 256, 256, 32)   4704        input_1[0][0]                    
__________________________________________________________________________________________________
bias (Bias)                     (64, 256, 256, 32)   2097152     conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (64, 256, 256, 32)   0           bias[0][0]                       
_________________________________________________________________________________

## Loading weights

In [17]:
with open('../models/JFNET_weights.pickle', 'r', encoding='bytes') as handle:
    jfnet_weights = pickle.load(handle)

LookupError: unknown encoding: bytes

In [ ]:
model.layers[1].get_weights()[1]

In [ ]:
x=concatenate([tf.zeros((1, 1)), tf.zeros((1, 1))])
x.get_shape()

In [ ]:
help(tf.TensorShape)

# Load model from pickle

build_model().summary()